In [ ]:
import cv2
if __name__ == '__main__' :

    video = cv2.VideoCapture("C:/Users/umar.amanat/Desktop/Mot16-Venice-2.mp4");
    
    # Find OpenCV version
    (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
    
    if int(major_ver)  < 3 :
        fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
        print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
    else :
        fps = video.get(cv2.CAP_PROP_FPS)
        print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
    
    video.release(); 

#### Calculating WIdth and Height of frame

In [ ]:
import cv2

vcap = cv2.VideoCapture("C:/Users/umar.amanat/Desktop/Mot16-Venice-2.mp4");

if vcap.isOpened(): 
    height = vcap.get(cv2.CAP_PROP_FRAME_HEIGHT)   # float
    width = vcap.get(cv2.CAP_PROP_FRAME_WIDTH)
    print("video height {} and width {}".format(height, width))


#### Saving video frame after skipping particular number of frames

In [ ]:
import os 
import cv2

vid_fold_path = "C:/Users/umar.amanat/Desktop" #Video folder path
video_path = "MOT16-11.mp4"

vidcap = cv2.VideoCapture(os.path.join(vid_fold_path, video_path))
total_frames = vidcap.get(7)
fps = vidcap.get(cv2.cv2.CAP_PROP_FPS)   #fps of video
success,image = vidcap.read() #read video
skip_frame = 5  # number of frame to skip
orig_height = vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)   # original height of video
orig_width = vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)     #original width of videos
print("frame per second: ", fps)
print("video height {} and width {}".format(orig_height, orig_width))
success = True    #flag for iterating through video's frame
frame_count = 0         #count for storing video file
scaled_height = int(orig_height/32) * 32  #scaled height for yolov2 algo
scaled_width = int(orig_width/32) * 32    #scaled width for yolov2 algo
print("scaled video height {} and scaled width {}".format(scaled_height, scaled_width))

image_count = 0 #counter for counting number of images per folder
folder_count = 0 #counter for counting number of folder
saving_path = "C:/Users/umar.amanat/Desktop/DataSet"


saving_path = os.path.join(saving_path, "folder_")
os.makedirs(saving_path+str(folder_count))

diff_height = 0
diff_widht = 0
#Iterate till last frame of video
while success:
    if (frame_count%skip_frame == 0):
        if(image_count<10):
            diff_height = int(orig_height/2)-int(scaled_height/2)
            diff_width = int(orig_width/2)-int(scaled_width/2)
            image = image[diff_height : int(orig_height-diff_height), diff_width : int(orig_width-diff_width)]
            cv2.imwrite(os.path.join(saving_path+str(folder_count), video_path+"_"+str(image_count))+".png" , image) 
            image_count += 1
        else:
            folder_count += 1;
            saving_path = saving_path #os.path.join(saving_path, "folder_"+str(folder_count))
            os.makedirs(saving_path+str(folder_count))
            image_count = 0
            
    success,image = vidcap.read()    
    frame_count += 1 

#### Brightening, Rotating Images

In [ ]:
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    r_img = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)  
    return r_img
    #rotate func end here


In [ ]:
def brightened_image(image, intensity):
    """
    Argument info
    image: input image
    intesnity: Intensity for brightness
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) #convert it to hsv
    for x in range(0, len(hsv)):
        for y in range(0, len(hsv[0])):
            if(hsv[x, y][2] < 255 - intensity):
                hsv[x, y][2] += intensity
    
    b_img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR) 
    return b_img
    #brightened image end here

In [ ]:
def increase_blue(image, intensity):
    """
    Argument info
    image: input image
    intesnity: Intensity for blue color
    """
    for x in range(0, len(image)):
        for y in range(0, len(image[0])):
            if(image[x, y][0] < 255 - intensity):
                image[x, y][0] += intensity
    return image
    #increase blue image end here

In [ ]:
def increase_green(image, intensity):
    """
    Argument info
    image: input image
    intesnity: Intensity for green color
    """
    for x in range(0, len(image)):
        for y in range(0, len(image[0])):
            if(image[x, y][1] < 255 - intensity):
                image[x, y][1] += intensity
    return image
    #increase green image end here

In [ ]:
def increase_red(image, intensity):
    """
    Argument info
    image: input image
    intesnity: Intensity for green color
    """
    for x in range(0, len(image)):
        for y in range(0, len(image[0])):
            if(image[x, y][2] < 255 - intensity):
                image[x, y][2] += intensity
    return image
    #increase red image end here

In [ ]:
def apply_kernel(image, kernel):
    """
    Image: input image
    Kernel : Specify 2D kernel to apply on image 
    """
    image = cv2.filter2D(image, -1, kernel)
    return image


In [ ]:
def increase_contrast(image):
    clahe = cv2.createCLAHE(clipLimit=3., tileGridSize=(8,8))
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)  # convert from BGR to LAB color space
    l, a, b = cv2.split(lab)  # split on 3 different channels
    l2 = clahe.apply(l)  # apply CLAHE to the L-channel
    lab = cv2.merge((l2,a,b))  # merge channels
    img2 = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)  # convert from LAB to BGR
    return img2


In [ ]:
def adding_noise(image):
    row,col,ch = image.shape
    s_vs_p = 0.5
    amount = 0.004
    out = np.copy(image)
    # Salt mode
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt))
          for i in image.shape]
    out[coords] = 1

    # Pepper mode
    num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper))
          for i in image.shape]
    out[coords] = 0
    return out

In [ ]:
import os 
import cv2
import numpy as np
import time
import glob

img_fold_path = "C:/Users/umar.amanat/Desktop/Images" #Video folder path
saving_path = "C:/Users/umar.amanat/Desktop/TunedImages"
img_cnt = 0 #image_count
angle = -0.5    #angle for rotation
br_int = 35     #brightness intensity
b_int = 35      #blue color intensity
g_int = 35      #green intensity
r_int = 35      #red intensity

os.makedirs(saving_path)
for image_path in glob.iglob(os.path.join(img_fold_path, "*.png")):
    image = cv2.imread(image_path) #reading image
    i_n = image_path.split("\\")[-1].split(".")[0]    #extract image name from image path
    t1 = time.time()
    ######################## make image bright start ######################################
    br_img = brightened_image(image.copy(), br_int)
    print("Brightened image {} in {} seconds: ".format(image_path, time.time()-t1))
    cv2.imwrite(os.path.join(saving_path, i_n+"_bright_"+str(img_cnt))+".png", br_img) 
    ######################## end######################################

    t1 = time.time()    
    ######################## Rotate image start ######################################
    rot_img = rotate_image(image.copy(), angle)    #rotated image
    print("Rotate image {} in {} seconds: ".format(image_path, time.time()-t1))
    cv2.imwrite(os.path.join(saving_path, i_n+"_rotate_"+str(img_cnt))+".png", rot_img) 
    ######################## end ######################################
    
    t1 = time.time()    
    ######################## increase blue intensity start ######################################
    blue_img = increase_blue(image.copy(), b_int)    
    print("increase blue color {} in {} seconds: ".format(image_path, time.time()-t1))
    cv2.imwrite(os.path.join(saving_path, i_n+"_blue_"+str(img_cnt))+".png", blue_img) 
    ######################## end ######################################
    
    t1 = time.time()    
    ######################## increase green intensity start ######################################
    green_img = increase_green(image.copy(), g_int)    
    print("increase green color {} in {} seconds: ".format(image_path, time.time()-t1))
    cv2.imwrite(os.path.join(saving_path, i_n+"_green_"+str(img_cnt))+".png", green_img) 
    ######################## end ######################################
    
    
    t1 = time.time()    
    ######################## increase green intensity start ######################################
    red_img = increase_red(image.copy(), g_int)    
    print("increase red color {} in {} seconds: ".format(image_path, time.time()-t1))
    cv2.imwrite(os.path.join(saving_path, i_n+"_red_"+str(img_cnt))+".png", red_img) 
    ######################## end ######################################

    t1 = time.time()    
    ######################## image sharpening ######################################
    sharp_kernel = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
    sh_img = apply_kernel(image.copy(), sharp_kernel)    #sharp image
    print("Sharpened image {} in {} seconds: ".format(image_path, time.time()-t1))
    cv2.imwrite(os.path.join(saving_path, i_n+"_sharp_"+str(img_cnt))+".png", sh_img) 
    ######################## end ######################################
    
    t1 = time.time()
    ######################## increase contrast ######################################
    inc_cont = increase_contrast(image.copy())
    print("Increase contrast {} in {} seconds: ".format(image_path, time.time()-t1))
    cv2.imwrite(os.path.join(saving_path, i_n+"_contrast_"+str(img_cnt))+".png", inc_cont) 
    ######################## end ######################################
    
    
    t1 = time.time()
    ######################## Adding noise in images ######################################
    noise_img = adding_noise(image.copy())
    print("Adding noise in image {} in {} seconds: ".format(image_path, time.time()-t1))
    cv2.imwrite(os.path.join(saving_path, i_n+"_noise_"+str(img_cnt))+".png", noise_img) 
    ######################## end ######################################
   
    img_cnt+=1
    
print("Augmentation completed")

## Following script are only for flipped of annotated images

##### flipping annotated images

In [ ]:
import pandas as pd
import json
import os 
import cv2
import glob



img_fold_path = "C:/Users/umar.amanat/Desktop/Images" #images folder path
saving_path = "C:/Users/umar.amanat/Desktop/Flipped_Images"  #saving path for images
ann_svg_path = "C:/Users/umar.amanat/Desktop/Flipped_Images"   #path for saving annotation


img_ann_path = "C:/Users/umar.amanat/Desktop/Images/via_region_data.csv"  #path for annotated
ann_csv = pd.read_csv(img_ann_path)
labels = ann_csv["#filename"] 
tuples = list(zip(labels, ann_csv.index.values))
hier_index = pd.MultiIndex.from_tuples(tuples)
ann_csv.set_index(hier_index, inplace=True)
ann_csv.drop(columns=["#filename", "file_attributes", "region_attributes"], inplace=True)

img_cnt = 0
os.makedirs(saving_path)
for image_path, image_name in zip(glob.iglob(os.path.join(img_fold_path, "*.png")), ann_csv.index.levels[0]):
    image = cv2.imread(image_path) #reading image
    
     ####################### Flipping image ######################################
    hor_flip = cv2.flip( image.copy(), 1 )
#     print("Flipping image {} in {} seconds: ".format(image_path, time.time()-t1))
    cv2.imwrite(os.path.join(saving_path, image_path.split("\\")[-1].split(".")[0]+"_flip_"+str(img_cnt))+".png", hor_flip) 
    ######################## end ######################################
    
    #################FLipping of annotation start here4 ################
    for box in ann_csv.loc[image_name].index:
        dim = ann_csv.loc[image_name].loc[box]["region_shape_attributes"]
        json_string = json.loads(dim)
        json_string['x'] = image.shape[1] - json_string['x'] - json_string['width']
        ann_csv.replace(to_replace=dim, value=str(json_string), inplace=True)
        ann_csv.loc[image_name].loc[box]["region_shape_attributes"] = str(json_string)
    
    #######################end #####################
    img_cnt+=1

    #for end here
#major for end here    

#write annotation to new csv file
ann_csv.to_csv(os.path.join(ann_svg_path, "flip_annotation.csv"))

#### Shift annotated images

In [ ]:
import pandas as pd
import json
import os 
import cv2
import glob
import numpy as np



img_fold_path = "C:/Users/umar.amanat/Desktop/Images" #images folder path
saving_path = "C:/Users/umar.amanat/Desktop/shifted_images"  #saving path for images
ann_svg_path = "C:/Users/umar.amanat/Desktop/shifted_images"   #path for saving annotation


img_ann_path = "C:/Users/umar.amanat/Desktop/Images/via_region_data.csv"  #path for annotated
ann_csv = pd.read_csv(img_ann_path)
labels = ann_csv["#filename"] 
tuples = list(zip(labels, ann_csv.index.values))
hier_index = pd.MultiIndex.from_tuples(tuples)
ann_csv.set_index(hier_index, inplace=True)
ann_csv.drop(columns=["#filename", "file_attributes", "region_attributes"], inplace=True)

x_shift = 10    #shifting along x-axis
y_shift = 10    #shifting along y-axis

img_cnt = 0
os.makedirs(saving_path)
for image_path, image_name in zip(glob.iglob(os.path.join(img_fold_path, "*.png")), ann_csv.index.levels[0]):
    image = cv2.imread(image_path) #reading image
     ####################### Flipping image ######################################
    num_rows, num_cols = image.copy().shape[:2]
    translation_matrix = np.float32([ [1,0,x_shift], [0,1,y_shift] ])
    shf_img = cv2.warpAffine(image.copy(), translation_matrix, (num_cols, num_rows))
#     shf_img = cv2.copyMakeBorder(shf_img,y_shift*10,0,x_shift*10,10,cv2.BORDER_REFLECT_101)
    cv2.imwrite(os.path.join(saving_path,image_path.split("\\")[-1].split(".")[0]+ "_shift_"+str(img_cnt))+".png", shf_img) 
    ######################## end ######################################
    
    #################FLipping of annotation start here ################
    for box in ann_csv.loc[image_name].index:
        dim = ann_csv.loc[image_name].loc[box]["region_shape_attributes"]
        json_string = json.loads(dim)
        json_string['x'] = json_string['x'] + x_shift 
        json_string['y'] = json_string['y'] + y_shift
        ann_csv.replace(to_replace=dim, value=str(json_string), inplace=True)
        ann_csv.loc[image_name].loc[box]["region_shape_attributes"] = str(json_string)
    
    #######################end #####################
    #inner for end here
    img_cnt+=1
#major for end here    

#write annotation to new csv file
ann_csv.to_csv(os.path.join(ann_svg_path, "shift_annotation.csv"))

### Scripts for overlay transparent image on other background image

In [69]:
import random
def transparentOverlay(src , overlay , pos=(0,0),scale = 1):
    """
    :param src: Input Color Background Image
    :param overlay: transparent Image (BGRA)
    :param pos:  position where the image to be blit.
    :param scale : scale factor of transparent image.
    :return: Resultant Image
    """
    overlay = cv2.resize(overlay,(0,0),fx=scale,fy=scale)
    h,w,_ = overlay.shape  # Size of foreground
    rows,cols,_ = src.shape  # Size of background Image
    y,x = pos[0],pos[1]    # Position of foreground/overlay image
    y = random.randint(y, y+20)
    x = random.randint(x, x+20)
    #loop over all pixels and apply the blending equation
    for i in range(h):
        for j in range(w):
            if x+i >= rows or y+j >= cols:
                continue
            alpha = float(overlay[i][j][3]/255.0) # read the alpha channel 
            src[x+i][y+j] = alpha*overlay[i][j][:3]+(1-alpha)*src[x+i][y+j]
    return src


In [78]:
import cv2
import os
import random 
import pandas as pd
import glob
import json

img_fold_path = "C:/Users/umar.amanat/Desktop/Images" #images folder path
saving_path = "C:/Users/umar.amanat/Desktop/OverlayImages"  #saving path for images
bag_image_path = "C:/Users/umar.amanat/Desktop/shopping_bag_images"

img_ann_path = "C:/Users/umar.amanat/Desktop/Images/via_region_data.csv"  #path for annotated
ann_csv = pd.read_csv(img_ann_path)
labels = ann_csv["#filename"] 
tuples = list(zip(labels, ann_csv.index.values))
hier_index = pd.MultiIndex.from_tuples(tuples)
ann_csv.set_index(hier_index, inplace=True)
ann_csv.drop(columns=["#filename", "file_attributes", "region_attributes"], inplace=True)

img_cnt=0
bag_img = cv2.imread(os.path.join(bag_image_path, "world_cup.png"), cv2.IMREAD_UNCHANGED)

# if(bag_img.shape[0] > 400 & bag_img.shape[1] > 400):
#     bag_img = cv2.resize(bag_img, (400, 400))

os.makedirs(saving_path)
for image_path, image_name in zip(glob.iglob(os.path.join(img_fold_path, "*.png")), ann_csv.index.levels[0]):
    per_img = cv2.imread(image_path)
    for box in ann_csv.loc[image_name].index:
        dim = ann_csv.loc[image_name].loc[box]["region_shape_attributes"]
        json_string = json.loads(dim)
        
        #generate random size of bag 
        bag_img = cv2.resize(bag_img, (int(json_string['width']*random.randint(35, 40)/100), 
                                       int(json_string['height']*random.randint(35, 40)/100))
                            , cv2.INTER_CUBIC)
#         bag_x = random.randint(json_string['x'], json_string['x'] + bag_img.shape[1])
        bag_y = json_string['y'] + json_string['height'] - int(bag_img.shape[0]*1.5)
#         bag_y = random.randint(bag_y - 30, bag_y)


#         per_img[bag_y:bag_y+bag_img.shape[0], bag_x:bag_x+bag_img.shape[1]] = bag_img
        per_img = transparentOverlay(per_img, bag_img, pos=(json_string['x'], bag_y))
        
    #######################end #####################
    #inner for end here
    cv2.imwrite(os.path.join(saving_path,image_path.split("\\")[-1].split(".")[0]+ "_pwb_"+str(img_cnt))+".png", per_img) 
    print("Processed image {}".format(img_cnt))
    img_cnt+=1

Processed image 0
Processed image 1


#### Script for handling transparency issues

In [24]:
import numpy as np
import cv2
import os 

# ==============================================================================

img_fold_path = "C:/Users/umar.amanat/Desktop/Images" #images folder path
bag_image_path = "C:/Users/umar.amanat/Desktop/shopping_bag_images"


# We load the images
l_img= cv2.imread(os.path.join(img_fold_path, "7.png"))
s_img = cv2.imread(os.path.join(bag_image_path, "world_cup.png"), cv2.IMREAD_UNCHANGED) # Load with transparency

s_img = cv2.resize(s_img, (500, 500))
def transparentOverlay(src , overlay , pos=(0,0),scale = 1):
    """
    :param src: Input Color Background Image
    :param overlay: transparent Image (BGRA)
    :param pos:  position where the image to be blit.
    :param scale : scale factor of transparent image.
    :return: Resultant Image
    """
    overlay = cv2.resize(overlay,(0,0),fx=scale,fy=scale)
    h,w,_ = overlay.shape  # Size of foreground
    rows,cols,_ = src.shape  # Size of background Image
    y,x = pos[0],pos[1]    # Position of foreground/overlay image
    
    #loop over all pixels and apply the blending equation
    for i in range(h):
        for j in range(w):
            if x+i >= rows or y+j >= cols:
                continue
            alpha = float(overlay[i][j][3]/255.0) # read the alpha channel 
            src[x+i][y+j] = alpha*overlay[i][j][:3]+(1-alpha)*src[x+i][y+j]
    return src


result = transparentOverlay(l_img, s_img)
cv2.imshow('img', result)
cv2.waitKey(0)
cv2.destroyAllWindows()
# cv2.imwrite("merged_transparent.png", result_2)

In [23]:
s_img = cv2.imread(os.path.join(bag_image_path, "world.png"), cv2.IMREAD_UNCHANGED) # Load with transparency
s_img.shape

(215, 234, 3)

In [ ]:
import cv2

img_fold_path = "C:/Users/umar.amanat/Desktop/Images/img_1.jpg" #Video folder path
image = cv2.imread(img_fold_path)
cv2.imshow('1', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
 t1 = time.time()    
    ######################## Unsharp Masking ######################################
    sharp_kernel = -1./256 * np.array([[1, 4, 6, 4, 1], 
                                       [4, 16, 24, 16, 4], 
                                       [6, 24, -476, 25, 6],
                                       [4, 16, 24, 16, 4],
                                       [1, 4, 6, 4, 1]
                                      ])
    un_sh_mask = apply_kernel(image.copy(), sharp_kernel)    #sharp image
    print("Unsharp masking image {} in {} seconds: ".format(image_path, time.time()-t1))
    cv2.imwrite(os.path.join(saving_path, "un_sh_mk_"+str(img_cnt))+".png", un_sh_mask) 
    ######################## end ######################################
    

In [ ]:
kernel.shape